In [1]:
!git clone https://github.com/saforem2/wordplay.git

fatal: destination path 'wordplay' already exists and is not an empty directory.


In [2]:
!python3 wordplay/data/shakespeare_char/prepare.py

Using HF_DATASETS_CACHE=/content/wordplay/data/shakespeare_char/.cache/huggingface
length of dataset in characters: 1,115,394
all the unique characters: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocab size: 65
train has 1,003,854 tokens
val has 111,540 tokens


In [3]:
!pip install tiktoken
!pip install enrich

In [4]:
!python3 wordplay/data/shakespeare/prepare.py

Using HF_DATASETS_CACHE=/content/wordplay/data/shakespeare/.cache/huggingface
train has 301,966 tokens
val has 36,059 tokens


In [5]:
!python3 -m pip install deepspeed

In [6]:
!python3 -m pip install -e wordplay

Obtaining file:///content/wordplay
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Cloning https://github.com/saforem2/ambivalent to /tmp/pip-install-o7jaqcjb/ambivalent_13c730ac4a0a4f2e997ac92ffccbb303
  Running command git clone --filter=blob:none --quiet https://github.com/saforem2/ambivalent /tmp/pip-install-o7jaqcjb/ambivalent_13c730ac4a0a4f2e997ac92ffccbb303
  Resolved https://github.com/saforem2/ambivalent to commit eac43ada80b6d4b2f71bf45cee9329993f622e87
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/saforem2/ezpz to /tmp/pip-install-o7jaqcjb/ezpz_7e42a159456f43e683c4c0687ae6f879
  Running command git clone --filter=blob:none --quiet https://github.com/saforem2

In [7]:
import os
import numpy as np
from ezpz import setup
from hydra.utils import instantiate
from wordplay.configs import get_config, PROJECT_ROOT
from wordplay.trainer import Trainer

HF_DATASETS_CACHE = PROJECT_ROOT.joinpath('.cache', 'huggingface')
HF_DATASETS_CACHE.mkdir(exist_ok=True, parents=True)

os.environ['HF_DATASETS_CACHE'] = HF_DATASETS_CACHE.as_posix()

BACKEND = 'DDP'

rank = setup(
    framework='pytorch',
    backend=BACKEND,
    seed=1234,
)

cfg = get_config(
    [
        'data=shakespeare',
        'model=shakespeare',
        'optimizer=shakespeare',
        'train=shakespeare',
        f'train.backend={BACKEND}',
        'train.compile=false',
        'train.dtype=float16',
        'train.max_iters=1000',
        'train.log_interval=50',
        'train.eval_interval=500',
    ]
)
config = instantiate(cfg)

[2024-11-06 21:15:54.991573][INFO][configs.py:81] - Setting HF_DATASETS_CACHE to /content/wordplay/.cache/huggingface/datasets

[2024-11-06 21:15:55.119459][INFO][dist.py:92] - 

[dist_info]:
  • DEVICE=cuda
  • DEVICE_ID=cuda:0
  • DISTRIBUTED_BACKEND=nccl
  • GPUS_PER_NODE=1
  • HOSTS=['6944be8b249d']
  • HOSTFILE=/content/hostfile
  • HOSTNAME=6944be8b249d
  • LOCAL_RANK=0
  • MACHINE=6944be8b249d
  • NUM_NODES=1
  • NGPUS=1
  • NGPUS_AVAILABLE=1
  • NODE_ID=0
  • RANK=0
  • SCHEDULER=LOCAL
  • WORLD_SIZE_TOTAL=1
  • WORLD_SIZE_IN_USE=1
  • LAUNCH_CMD=None

[2024-11-06 21:15:55.128405][INFO][dist.py:728] - [0/1] Using device='cuda' with backend='DDP' + 'nccl' for distributed training.

[2024-11-06 21:15:55.136186][INFO][dist.py:348] - [device='cuda'][rank=0/0][local_rank=0/0][node=0/0]

[2024-11-06 21:15:55.140240][WARNING][dist.py:352] - Using [1 / 1] available "cuda" devices !!

[2024-11-06 21:15:55.425071][INFO][configs.py:317] - Loading train from /content/wordplay/data/shakespeare_char/train.bin

[2024-11-06 21:15:55.432963][INFO][configs.py:317] - Loading val from /content/wordplay/data/shakespeare_char/val.bin

[2024-11-06 21:15:55.447649][INFO][configs.py:442] - Tokens per iteration: 16,384

[2024-11-06 21:15:55.450491][INFO][configs.py:465] - Using self.ptdtype=torch.float16 on self.device_type='cuda'

[2024-11-06 21:15:55.454313][INFO][configs.py:471] - Initializing a new model from scratch

In [8]:
trainer = Trainer(config)

[2024-11-06 21:15:55.469764][INFO][trainer.py:248] - Initializing a new model from scratch

[2024-11-06 21:15:56.237170][INFO][model.py:255] - number of parameters: 10.65M

[2024-11-06 21:15:56.360658][INFO][trainer.py:266] - Model size: num_params=10646784

[2024-11-06 21:15:56.364434][INFO][model.py:445] - num decayed parameter tensors: 26, with 10,740,096 parameters

[2024-11-06 21:15:56.367718][INFO][model.py:449] - num non-decayed parameter tensors: 13, with 4,992 parameters

...

[2024-11-06 21:15:57.434329][CRITICAL][trainer.py:318] - "devid='cuda:0'"

[2024-11-06 21:15:57.441789][INFO][trainer.py:358] - • self.model=GPT(
  (transformer): ModuleDict(
    (wte): Embedding(65, 384)
    (wpe): Embedding(256, 384)
    (drop): Dropout(p=0.2, inplace=False)
    (h): ModuleList(
      (0-5): 6 x Block(
        (ln_1): LayerNorm()
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=384, out_features=1152, bias=False)
          (c_proj): Linear(in_features=384, out_features=384, bias=False)
          (attn_dropout): Dropout(p=0.2, inplace=False)
          (resid_dropout): Dropout(p=0.2, inplace=False)
        )
        (ln_2): LayerNorm()
        (mlp): MLP(
          (c_fc): Linear(in_features=384, out_features=1536, bias=False)
          (act_fn): GELU(approximate='none')
          (c_proj): Linear(in_features=1536, out_features=384, bias=False)
          (dropout): Dropout(p=0.2, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm()
  )
  (lm_head): Linear(in_features=384, out_features=65, bias=False)
)

[2024-11-06 21:15:57.450962][INFO][trainer.py:359] - • self.grad_scaler=<torch.cuda.amp.grad_scaler.GradScaler object at 0x7cc561fbef20>

[2024-11-06 21:15:57.454821][INFO][trainer.py:360] - • self.model_engine=GPT(
  (transformer): ModuleDict(
    (wte): Embedding(65, 384)
    (wpe): Embedding(256, 384)
    (drop): Dropout(p=0.2, inplace=False)
    (h): ModuleList(
      (0-5): 6 x Block(
        (ln_1): LayerNorm()
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=384, out_features=1152, bias=False)
          (c_proj): Linear(in_features=384, out_features=384, bias=False)
          (attn_dropout): Dropout(p=0.2, inplace=False)
          (resid_dropout): Dropout(p=0.2, inplace=False)
        )
        (ln_2): LayerNorm()
        (mlp): MLP(
          (c_fc): Linear(in_features=384, out_features=1536, bias=False)
          (act_fn): GELU(approximate='none')
          (c_proj): Linear(in_features=1536, out_features=384, bias=False)
          (dropout): Dropout(p=0.2, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm()
  )
  (lm_head): Linear(in_features=384, out_features=65, bias=False)
)

[2024-11-06 21:15:57.463823][INFO][trainer.py:361] - • self.optimizer=AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.99)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: True
    lr: 0.001
    maximize: False
    weight_decay: 0.1

Parameter Group 1
    amsgrad: False
    betas: (0.9, 0.99)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: True
    lr: 0.001
    maximize: False
    weight_decay: 0.0
)

In [9]:
query = "Is all that glisters gold?"
outputs = trainer.evaluate(
    query,
    num_samples=1,
    max_new_tokens=256,
    top_k=16,
    display=False
)
print(f"['prompt']: '{query}'")
print("['response']:\n\n" + fr"{outputs['0']['raw']}")

['prompt']: 'Is all that glisters gold?'
['response']:

Is all that glisters gold?CbaJYf;;bbssEYiaP:s:iSMNKK3'ss$q KMSSSCSEEEa''iCfcHHaUtPSBBMMMIK'-YsAqC
QbCHGPcIIppEjHMvAr''YnMTT'MTTHs'A
CiCCiJJJfbbbbssbSvPVbiD-QHBQsQIaabbbSiIIMH''iPBKKQ-a-bs?JnVVVa  ibMMVWJJYCBEq
asbabbssff:wsVV?b'YVVYKQQ--iaaa?'bywYN.QpAV?KJjHEwwibbF--IHkkwKK KEE?iaa


In [10]:
trainer.config.device_type

'cuda'

In [11]:
trainer.train(train_iters=500)

                Training Legend                 
┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        abbr ┃ desc                           ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        step │ Current training iteration     │
│        loss │ Loss value                     │
│          dt │ Elapsed time per training step │
│         dtf │ Elapsed time per forward step  │
│         dtb │ Elapsed time per backward step │
│         sps │ Samples per second             │
│ sps_per_gpu │ Samples per second (per GPU)   │
│         tps │ Tokens per second              │
│ tps_per_gpu │ Tokens per second (per GPU)    │
│         mfu │ Model flops utilization        │
└─────────────┴────────────────────────────────┘

[2024-11-06 21:16:00.518947][INFO][trainer.py:827] - ['prompt']: 'What is an LLM?'

[2024-11-06 21:16:00.521290][INFO][trainer.py:831] - ['response']:

What is an LLM?:IQIi-VV'V-'peHQK'sQVbHcpAibsalHbbbbEGffSH:sB:QVhRVSVb--Z
ZKMiI:sff:MHVbbVbbdd-ZbkiJHbS:yaafIKHHHS'la'hbs?wVVSCCiiX'aqsb-ZvQJJHHb''-hsHHaiY,EEaUV',qCeb
oLlZpIIjppbbJf-KayyC:vHHa KJJ
'CrVY::G:sbbaf:sQhipNt,
'sVYipJpYC'SKiq$$iSSMsaacPa,QbIKttKK--wFbFFaiVEiii

[2024-11-06 21:16:17.822896][INFO][trainer.py:892] - step=50 loss=2.50675 dt=0.0969169 dtf=0.00762595 dtb=0.0111565 sps=10.3181 sps_per_gpu=10.3181 tps=169052 tps_per_gpu=169052 mfu=3.84477

[2024-11-06 21:16:22.619149][INFO][trainer.py:892] - step=100 loss=2.48042 dt=0.0878731 dtf=0.00421082 dtb=0.00677434 sps=11.38 sps_per_gpu=11.38 tps=186451 tps_per_gpu=186451 mfu=3.88434

...

[2024-11-06 21:16:42.057262][INFO][trainer.py:892] - step=300 loss=1.91912 dt=0.0992465 dtf=0.00541503 dtb=0.012366 sps=10.0759 sps_per_gpu=10.0759 tps=165084 tps_per_gpu=165084 mfu=3.84638

[2024-11-06 21:16:46.998530][INFO][trainer.py:892] - step=350 loss=1.79642 dt=0.1006 dtf=0.00474861 dtb=0.00704457 sps=9.94036 sps_per_gpu=9.94036 tps=162863 tps_per_gpu=162863 mfu=3.83214

[2024-11-06 21:16:51.965404][INFO][trainer.py:892] - step=400 loss=1.73325 dt=0.103738 dtf=0.00636424 dtb=0.0100203 sps=9.63963 sps_per_gpu=9.63963 tps=157936 tps_per_gpu=157936 mfu=3.80812

[2024-11-06 21:16:56.984955][INFO][trainer.py:892] - step=450 loss=1.67691 dt=0.0995576 dtf=0.00416727 dtb=0.00697224 sps=10.0444 sps_per_gpu=10.0444 tps=164568 tps_per_gpu=164568 mfu=3.80159

[2024-11-06 21:17:02.043721][INFO][trainer.py:892] - step=500 loss=1.60431 dt=0.100273 dtf=0.00429319 dtb=0.00712171 sps=9.97274 sps_per_gpu=9.97274 tps=163393 tps_per_gpu=163393 mfu=3.79304

In [13]:
import time

query = "Is all that glisters gold?"
t0 = time.perf_counter()
outputs = trainer.evaluate(
    query,
    num_samples=1,
    max_new_tokens=256,
    top_k=16,
    display=False
)
print(f'took: {time.perf_counter() - t0:.4f}s')
print(f"['prompt']: '{query}'")
print("['response']:\n\n" + fr"{outputs['0']['raw']}")

took: 1.6847s
['prompt']: 'Is all that glisters gold?'
['response']:

Is all that glisters gold?

TYBRAKE:
I have chend you and we there temption but that a crown bid
And double dead well her him of my glings,
And bece of his nighter mark of a down,
And for miness so.

GLOUCESTER:

KING RICHARD IV:
My me?

GLOUCESTER:
And it stam, be comes in this re
